In [38]:
# PNL decomposition
from datetime import datetime, date
import numpy as np
import pandas as pd
from io import StringIO

import plotly.graph_objects as go
import plotly.colors as pc

def evaluate_expression(df, expression):
    """
    Helper function to evaluate an expression string.
    The expression can reference columns in the DataFrame and perform operations on them.
    """
    return eval(expression, {'df': df})

def plot_cumulative_sum(df, column_expressions=None):
    """
    Function to plot cumulative sum of all columns in a time series dataframe with detailed hover info.
    Allows creation of new columns based on dictionary expressions.
    
    :param df: DataFrame with time as index and columns representing categories to sum cumulatively
    :param column_expressions: Dictionary of new column definitions (e.g., {'new_col': 'col1 + 0.5 * col2'})
    """
    # If there are new column expressions, create the new columns
    if column_expressions:
        for new_col, expression in column_expressions.items():
            df[new_col] = evaluate_expression(df, expression)
    
    # Initialize a figure
    fig = go.Figure()

    # Generate a list of colors using Plotly's color scale
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
            '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
            '#FF3F3F', '#FFD700', '#6495ED', '#4CAF50', '#D2691E', '#EE82EE', '#FFA500', '#32CD32', '#FF1493', '#4169E1',
            '#AEC6CF', '#FFB347', '#FF6961', '#77DD77', '#F49AC2', '#FFD1DC', '#B39EB5', '#FFDAE9', '#CFCFC4', '#E0E4CC',
            '#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e', '#e6ab02', '#a6761d', '#666666',
            '#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999',
            '#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#ffd92f', '#e5c494', '#b3b3b3',
            '#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5', '#ffed6f',
            '#7fc97f', '#beaed4', '#fdc086', '#ffff99', '#386cb0', '#f0027f', '#bf5b17']

    num_colors = len(colors)

    # Iterate through all columns (excluding 'Total' if it exists) and assign a unique color
    for i, column in enumerate(df.columns):
        if column != 'Total':
            # Compute cumulative sum of each column
            cumulative_sum = df[column].cumsum()

            # Add line plot for each column with hover info and unique color
            fig.add_trace(go.Scatter(
                x=df.index, 
                y=cumulative_sum, 
                mode='lines', 
                name=column,
                hovertemplate=(
                    f"<b>{column}</b><br>" +  # Column name
                    "Date: %{x}<br>" +        # Date
                    "Value: %{y:.2f}<extra></extra>"  # Cumulative value
                ),
                line=dict(color=colors[i % num_colors])  # Assign a color from the palette
            ))

    # Set title and labels with tight layout
    fig.update_layout(
        title="Cumulative Sum of All Columns Over Time",
        xaxis_title="Date",
        yaxis_title="Cumulative Sum",
        hovermode="x unified",
        margin=dict(l=40, r=40, t=50, b=40),  # Tight margins
        autosize=True,
        template='plotly_white',  # Clean template
        height=500,  # Set height
        width=900,   # Set width
    )

    # Show the plot
    fig.show()

def parse_pnl_string(csv_data):
    data = StringIO(csv_data)
    df = pd.read_csv(data, sep='\t', header=None, names=['key', 'value'], thousands=',')
    return df


In [17]:
pnl = {date(2024, 10, 1):  """
TA:EUR:IR	64,924
TA:GBP:IR	-6,007
TA:CHF:FX	19,549
TA:JPY:IR	118,263
TA:AUD:IR	137
TA:FX:DELTA	-20,304
TA:FX:VOL_RV	-208
TA:IR:CHF	65
TA:FX:USD	-86
TA:JPY:FX	-141
BROKER:ADJ	-4
CASH_USAGE	-251
TA:EQ:US	-69
TA:IR:GBP	1
TA:IR:USD:CURVE	24,935
TA:IR:ZER0_VAL	-8
TA:IR:ZERO_VAL	331
TA:KRW:IR	80
TA:USD:IR	-3,950
TA:USD:IR:HTM	444
TA:USD:IR:RV	29
USD SELLDOWN	4,073
BNP FXPB	-0
""", 
date(2024, 10, 2): """
TA:EUR:IR	120,062
TA:GBP:IR	708
TA:CHF:FX	19,783
TA:JPY:IR	-14,163
TA:AUD:IR	46
TA:FX:DELTA	-24,351
TA:FX:VOL_RV	-376
TA:IR:CHF	22
TA:FX:USD	62
TA:JPY:FX	-45
BROKER:ADJ	-1
CASH_USAGE	-235
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	-89,992
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	643
TA:KRW:IR	27
TA:USD:IR	60,887
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,356
BNP FXPB	-0""",
date(2024, 10, 3): """
TA:EUR:IR	1,376
TA:GBP:IR	-3,460
TA:CHF:FX	36,038
TA:JPY:IR	-31,639
TA:AUD:IR	46
TA:FX:DELTA	-37,997
TA:FX:VOL_RV	-2,647
TA:IR:CHF	22
TA:FX:USD	-17
TA:JPY:FX	-39
BROKER:ADJ	-1
CASH_USAGE	-235
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	28,763
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	295
TA:KRW:IR	27
TA:USD:IR	-133,548
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,355
BNP FXPB	-0
""", 
date(2024, 10, 4): """
TA:EUR:IR	-58,044
TA:GBP:IR	-58,517
TA:CHF:FX	15,207
TA:JPY:IR	-3,885
TA:AUD:IR	46
TA:FX:DELTA	-17,078
TA:FX:VOL_RV	-429
TA:IR:CHF	22
TA:FX:USD	22
TA:JPY:FX	-46
BROKER:ADJ	-1
CASH_USAGE	-235
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	22,196
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	280
TA:KRW:IR	27
TA:USD:IR	-84,925
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,356
BNP FXPB	-0
""",
date(2024, 10, 7): """
TA:EUR:IR	-11,432
TA:GBP:IR	20,850
TA:CHF:FX	-3,767
TA:JPY:IR	1,194
TA:AUD:IR	137
TA:FX:DELTA	2,550
TA:FX:VOL_RV	-317
TA:IR:CHF	65
TA:FX:USD	-114
TA:JPY:FX	-141
BROKER:ADJ	-4
CASH_USAGE	-25
TA:EQ:US	-69
TA:IR:GBP	1
TA:IR:USD:CURVE	16,234
TA:IR:ZER0_VAL	-8
TA:IR:ZERO_VAL	-3,454
TA:KRW:IR	80
TA:USD:IR	6,449
TA:USD:IR:HTM	443
TA:USD:IR:RV	29
USD SELLDOWN	4,068
BNP FXPB	-0
""",
date(2024, 10, 8): """
TA:EUR:IR	14,470
TA:GBP:IR	21,358
TA:CHF:FX	-40,397
TA:JPY:IR	33,882
TA:AUD:IR	137
TA:FX:DELTA	38,056
TA:FX:VOL_RV	495
TA:IR:CHF	65
TA:FX:USD	-132
TA:JPY:FX	-145
BROKER:ADJ	-4
CASH_USAGE	-218
TA:EQ:US	-69
TA:IR:GBP	1
TA:IR:USD:CURVE	114,397
TA:IR:ZER0_VAL	-8
TA:IR:ZERO_VAL	17,501
TA:KRW:IR	80
TA:USD:IR	36,185
TA:USD:IR:HTM	443
TA:USD:IR:RV	29
USD SELLDOWN	4,071
BNP FXPB	-0
""",
date(2024, 10, 9): """
TA:EUR:IR	-13,617
TA:GBP:IR	-185,625
TA:CHF:FX	17,102
TA:JPY:IR	-13,977
TA:AUD:IR	46
TA:FX:DELTA	-17,758
TA:FX:VOL_RV	-3
TA:IR:CHF	22
TA:FX:USD	-18
TA:JPY:FX	-46
BROKER:ADJ	-1
CASH_USAGE	-201
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	47,701
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	-10,772
TA:KRW:IR	27
TA:USD:IR	-14,668
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,356
BNP FXPB	-0
""",
date(2024, 10, 10): """
TA:EUR:IR	-43,621
TA:GBP:IR	-222,757
TA:CHF:FX	20,944
TA:JPY:IR	-3,874
TA:AUD:IR	46
TA:FX:DELTA	-20,676
TA:FX:VOL_RV	-939
TA:IR:CHF	22
TA:FX:USD	-6
TA:JPY:FX	-44
BROKER:ADJ	-1
CASH_USAGE	-201
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	-34,323
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	6,184
TA:KRW:IR	27
TA:USD:IR	-22,005
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,356
BNP FXPB	-0
""",
date(2024, 10, 11): """
TA:EUR:IR	-27,548
TA:GBP:IR	-234,880
TA:CHF:FX	-16,172
TA:JPY:IR	-11,949
TA:AUD:IR	46
TA:FX:DELTA	16,061
TA:FX:VOL_RV	232
TA:IR:CHF	22
TA:FX:USD	-14
TA:JPY:FX	-49
BROKER:ADJ	-1
CASH_USAGE	-201
TA:EQ:US	-23
TA:IR:GBP	0
TA:IR:USD:CURVE	32,481
TA:IR:ZER0_VAL	-3
TA:IR:ZERO_VAL	1,737
TA:KRW:IR	27
TA:USD:IR	-57,886
TA:USD:IR:HTM	148
TA:USD:IR:RV	10
USD SELLDOWN	1,358
BNP FXPB	-0
""",
date(2024, 10, 14): """
TA:EUR:IR	25,266
TA:GBP:IR	-150,764
TA:CHF:FX	-370
TA:JPY:IR	-468
TA:AUD:IR	137
TA:FX:DELTA	-1,038
TA:FX:VOL_RV	16
TA:IR:CHF	65
TA:FX:USD	-120
TA:JPY:FX	-142
BROKER:ADJ	-4
CASH_USAGE	-25
TA:EQ:US	-69
TA:IR:GBP	1
TA:IR:USD:CURVE	2,854
TA:IR:ZER0_VAL	-8
TA:IR:ZERO_VAL	8,382
TA:KRW:IR	80
TA:USD:IR	-14,074
TA:USD:IR:HTM	444
TA:USD:IR:RV	29
USD SELLDOWN	4,073
BNP FXPB	-0
""",
date(2024, 10, 15): """
TA:EUR:IR	-10,392
TA:GBP:IR	-169,341
TA:CHF:FX	58,623
TA:JPY:IR	403
TA:AUD:IR	137
TA:FX:DELTA	-57,841
TA:FX:VOL_RV	-742
TA:IR:CHF	65
TA:FX:USD	-85
TA:JPY:FX	-138
BROKER:ADJ	-4
CASH_USAGE	-25
TA:EQ:US	-69
TA:IR:GBP	1
TA:IR:USD:CURVE	15,695
TA:IR:ZER0_VAL	-8
TA:IR:ZERO_VAL	8,388
TA:KRW:IR	80
TA:USD:IR	-76,854
TA:USD:IR:HTM	444
TA:USD:IR:RV	29
USD SELLDOWN	4,074
BNP FXPB	-0
"""
}

In [39]:
pnl_dfs = {dt: parse_pnl_string(pnl[dt]) for dt in pnl.keys()}
list_of_dataframes = []

for date, df in pnl_dfs.items():
    # Set the key column as columns, the value column as data, and add date as index
    df.set_index('key', inplace=True)
    df_transposed = df.T  # Transpose to make keys as columns
    df_transposed['date'] = date  # Add the date as a column
    list_of_dataframes.append(df_transposed)  # Append to the list

# Concatenate all the dataframes into a single one, setting 'date' as the index
final_df = pd.concat(list_of_dataframes).set_index('date')
final_df['Total'] = final_df.sum(axis=1)

column_expressions = {
    'FX': 'df["TA:FX:DELTA"] + df["TA:FX:VOL_RV"] + df["TA:FX:USD"] + df["TA:JPY:FX"] + df["TA:CHF:FX"]',
    'USD': 'df["TA:IR:USD:CURVE"] + df["TA:USD:IR"] + df["TA:USD:IR:HTM"] + df["TA:USD:IR:RV"]',
}
plot_cumulative_sum(final_df, column_expressions)

In [37]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc

# Generate samples of qualitative and continuous color palettes
color_schemes = {
    'Plotly': pc.qualitative.Plotly,
    'D3': pc.qualitative.D3,
    'Bold': pc.qualitative.Bold,
    'Pastel': pc.qualitative.Pastel,
    'Dark2': pc.qualitative.Dark2,
    'Set3': pc.qualitative.Set3,
}

# Create a sample plot to visualize different palettes
fig = go.Figure()

# Add a scatter plot for each color scheme
for i, (name, colors) in enumerate(color_schemes.items()):
    # Generate a number of sample points to display the palette
    num_samples = min(len(colors), 20)  # Show up to 10 colors per palette
    x = [i] * num_samples
    y = list(range(num_samples))
    fig.add_trace(go.Scatter(
        x=x, 
        y=y, 
        mode='markers', 
        marker=dict(size=20, color=colors[:num_samples]), 
        name=name,
        showlegend=True
    ))

# Update the layout for better visibility
fig.update_layout(
    title="Sample of Color Palettes",
    xaxis_title="Palette Name",
    yaxis_title="Color Index",
    xaxis=dict(tickmode='array', tickvals=list(range(len(color_schemes))), ticktext=list(color_schemes.keys())),
    height=500,
    width=800,
    showlegend=True
)

# Show the plot
fig.show()